In [2]:
import pandas as pd
import dask.dataframe as dd
import pickle
from dask.delayed import delayed

In [3]:
HEADERS = [
    "Pool_ID",
    "Loan_ID",
    "Month_Reporting_Period",
    "Channel",
    "Seller_Name",
    "Servicer_Name",
    "Master_Servicer",
    "Original_Interest_Rate",
    "Current_Interest_Rate",
    "Original_UPB",
    "UPB_at_Issuance",
    "Current_Actual_UPB",
    "Original_Loan_Term",
    "Origination_Date",
    "First_Payment_Date",
    "Loan_Age",
    "Months_to_Legal_Maturity",
    "Months_to_Maturity",
    "Maturity_Date",
    "LTV",
    "CLTV",
    "Number_of_Borrowers",
    "DTI",
    "Borrower_Credit_Score",
    "Co-Borrower_Credit_Score",
    "First_Time_Home_Buyer",
    "Loan_Purpose",
    "Property_Type",
    "Number_of_Units",
    "Occupancy_Status",
    "Property_State",
    "MSA",
    "Zip",
    "Mortgage_Insurance_Per",
    "Amortization_Type",
    "Prepayment_Penalty_Indicator",
    "Interest_Only_Loan_Indicator",
    "First_Principal_Interest_Payment_date",
    "Months_to_Amortization",
    "Current_Loan_Delinquency_Status",
    "Loan_Payment_History",
    "Modification_Flag",
    "Mortgage_Insurance_Cancellation",
    "Zero_Balance_Code",
    "Zero_Balance_Effective_Date",
    "UPB_at_Time_Removal",
    "Repurchase_Date",
    "Scheduled_Principal_Current",
    "Total_Principal_Current",
    "Unscheduled_Principal_Current",
    "Last_Paid_Installment_Date",
    "Foreclosure_Date",
    "Disposition_Date",
    "Foreclosure_Costs",
    "Property_Preservation_And_Repair_Costs",
    "Asset_Recovery_Costs",
    "Miscellaneous_Holding_Expenses",
    "Associated_Taxes",
    "Net_Sales_Proceeds",
    "Credit_Enhancement_Proceeds",
    "Repurchase_Make_Whole_Proceeds",
    "Other_Foreclosure_Proceeds",
    "UPB",
    "Principal_Forgiveness_Amount",
    "Original_List_Start_Date",
    "Original_List_Price",
    "Current_List_Start_Date",
    "Current_List_Price",
    "Borrower_Credit_Score_At_Issuance",
    "Co-Borrower_Credit_Score_At_Issuance",
    "Borrower_Credit_Score_Current",
    "Co-Borrower_Credit_Score_Current",
    "Mortgage_Insurance_Type",
    "Servicing_Activity_Indicator",
    "Current_Period_Modification_Loss_Amount",
    "Cumulative_Modification_Loss_Amount",
    "Current_Period_Credit_Event",
    "Cumulative_Credit_Event",
    "HomeReady_Indicator",
    "Foreclosure_Principal_Write-off",
    "Relocation_Mortgage",
    "Zero_Balance_Code_Change_Date",
    "Loan_Holdback",
    "Loan_Holdback_Effective_Date",
    "Delinquent_Accrued",
    "Property_Valuation_Method",
    "High_Balance_Loan",
    "ARM_Period_<=5",
    "ARM_Product_Type",
    "Initial_Fixed-Rate_Period",
    "Interest_Rate_Adjustament_Frequency",
    "Next_Interest_Rate_Adjustment_Date",
    "Next_Payment_Change_Date",
    "Index",
    "ARM_Cap_Structure",
    "Initial_Interest_Rate_Cap_Up",
    "Periodic_Interest_Rate_Cap_Up",
    "Lifetime_Interest_Rate",
    "Mortgage_Margin",
    "ARM_Ballon",
    "ARM_Plan_Number",
    "Borrower_Assistance_Plan",
    "High_Loan_to_Value",
    "Deal_Name",
    "Repurchase_Make_Whole_Proceeds_Flag",
    "Delinquency_Resolution",
    "Delinquency_Resolution_Count",
    "Total_Deferral_Amount",
]

SELECT = [
    'Loan_ID',
    'Channel',
    'Seller_Name',
    'Original_Interest_Rate',
    'Original_UPB',
    'Original_Loan_Term',
    "Origination_Date",
    "First_Payment_Date",
    "LTV",
    "CLTV",
    "Number_of_Borrowers",
    "DTI",
    "Borrower_Credit_Score",
    "Co-Borrower_Credit_Score",
    "First_Time_Home_Buyer",
    "Loan_Purpose",
    "Property_Type",
    "Number_of_Units",
    "Occupancy_Status",
    "Property_State",
    "Zip",
    "Mortgage_Insurance_Per",
    
    "Foreclosure_Date",
]

In [4]:
import os
data_files = os.listdir('../data/raw/')

In [5]:
def count_loans():
    """ Return a dict with loan counts by ID """
    chunksize = 10 ** 6
    data = pd.DataFrame()
    counts_dict = {}
    for file in data_files:
        if file == '.gitkeep':
            continue
        for chunk in pd.read_csv('../data/raw/'+file, sep='|', header=None, names=HEADERS, usecols=SELECT, chunksize=chunksize):
            grouped = chunk.groupby('Loan_ID')['Foreclosure_Date'].count().reset_index()
            for row in grouped.itertuples():
                id_ = row.Loan_ID
                counts = row.Foreclosure_Date
                if id_ in counts_dict.keys():
                    counts_dict[id_] += counts
                else:
                    counts_dict[id_] = counts
        print(file)
    return counts_dict

In [38]:
import os
import zipfile

def count_loans():
    """ Return a dict with loan counts by ID """
    counts_dict = {}    
    for file in data_files:
        if file == '.gitkeep' or not file.startswith('2017'):
            continue
        file_path = os.path.join('../data/raw', file)
        with open(file_path, 'r') as f:
            for line in f:
                line_splited = line.split('|')
                loan_id, date = line_splited[1], line_splited[51]

                if loan_id in counts_dict.keys():
                    counts_dict[loan_id]['count'] += 1
                else:
                    counts_dict[loan_id] = {
                        'foreclosure_status': False,
                        'count': 1,
                    }
                if date != '':
                    counts_dict[loan_id]['foreclosure_status'] = True
        print(file)
    return counts_dict

In [39]:
%%time
#counts_dict = count_loans()
#pickle.dump(counts_dict, open('../data/interim/counts_dict.pkl', 'wb'))

2017Q1.csv
2017Q3.csv
2017Q4.csv
2017Q2.csv
CPU times: user 3min 12s, sys: 7.69 s, total: 3min 19s
Wall time: 3min 20s


In [8]:
counts_dict = pickle.load(open('../data/interim/counts_dict.pkl', 'rb'))

In [20]:
%%time
chunksize = 10 ** 6
data = pd.DataFrame()
for file in data_files:
    if file == '.gitkeep' or not file.startswith('2017'):
        continue
    for chunk in pd.read_csv('../data/raw/'+file, sep='|', header=None, names=HEADERS, usecols=SELECT, chunksize=chunksize):
        
        
        
        
        data = data.append(chunk.drop('Foreclosure_Date', axis=1))
        data = data.drop_duplicates()
        
    print(file, data.shape[0])

2017Q1.csv 595101
2017Q3.csv 1289858
2017Q4.csv 1961673
2017Q2.csv 2580856
CPU times: user 5min 29s, sys: 49.1 s, total: 6min 19s
Wall time: 6min 25s


In [46]:
def get_value(loan_id, key):
    return counts_dict[str(loan_id)][key]

In [45]:
counts_dict['100002130634']

{'foreclosure_status': False, 'count': 30}

In [47]:
data['count'] = data['Loan_ID'].apply(lambda loan_id: get_value(loan_id, 'count'))
data['foreclosure'] = data['Loan_ID'].apply(lambda loan_id: get_value(loan_id, 'foreclosure_status'))

In [51]:
data.to_csv('../data/interim/loans_interim.csv', index=False)

In [48]:
data['foreclosure'].value_counts()

False    2579502
True        1354
Name: foreclosure, dtype: int64